### 1. 패키지 가져오기

In [1]:
# 라이브러리 사전 설치(프로그램 실행 전 설치 필요)
# pip install numpy pandas scipy scikit-learn pyod tensorflow

## 액세스 거부 발생 시, "pip install [패키지명] --user" 실행

In [2]:
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf
import statsmodels.api as sm
from scipy.stats import shapiro, stats, wilcoxon
from sklearn.metrics import classification_report
warnings.filterwarnings('ignore')

print("TF Version : {}".format(tf.__version__))

TF Version : 2.3.0


### 2. Dataset 가져오기
- train, test 데이터셋을 각각 load 후 concatenate 진행

In [3]:
path = './'

train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
df = pd.concat([train, test])
df.reset_index(drop=True, inplace=True)

print('shape: {}'.format(df.shape))

shape: (9650, 100)


### 3. Label 이진화 수행
- 이진 분류를 위해 문자를 숫자 형태로 변경

In [4]:
# GOOD/NOT GOOD에 대하여 0과 1로 이진화 수행
state = np.where(df['RESULT'] == 'GOOD', 0, 1)  # 0: 정상 / 1: 불량
df = df.assign(RESULT=state)

### 4. 결측치 확인 및 제거

In [5]:
# 결측치 확인
df.isnull().sum()

LOT                 0
CycleTime           0
uvlamp_cycletime    0
InjTime             0
RecoveryT           0
                   ..
OUTHUMIDITY         0
MI_TEST_RESULT      0
PAMCALE_RESULT      0
TIME_STAMP          0
RESULT              0
Length: 100, dtype: int64

In [6]:
# 결측지 제거
print("결측치 제거 전 shape : {}".format(df.shape))
df.dropna(axis=1, inplace=True)
print("결측치 제거 후 shape : {}".format(df.shape))

결측치 제거 전 shape : (9650, 100)
결측치 제거 후 shape : (9650, 100)


### 5. Object type 제거
- 데이터를 모델에 학습시키기 위해서 object type을 제거해주어야 함.

In [7]:
print("--제거된 Object feature--")
for col in df.columns:
    if df[col].dtype == object:
        df.drop(columns=col, axis=1, inplace=True)
        print(col)

--제거된 Object feature--
LOT
TIME_STAMP


### 6. 유효한 속성만 추출
<h4>1. 회귀 분석 수행</h4>
    
- 잔차제곱합(RSS: Residual Sum of Squares)을 최소화하는 가중치 벡터를 구하는 방법
- 각각의 독립변수 x_{i}가 종속변수 y에 영향이 있는지 확인가능하며 p-value 값을 기준으로 무효한 feature들을 제거

In [8]:
# 전진 단계별 선택법
variables = df.columns[:-2].tolist() ## 설명 변수 리스트
selected_variables = []  ## 선택된 변수들 리스트
sl_enter = 0.05
sl_remove = 0.05

sv_per_step = []  ## 각 스텝별로 선택된 변수들
adjusted_r_squared = []  ## 각 스텝별 수정된 결정계수
steps = []
step = 0

while len(variables) > 0:
    remainder = list(set(variables) - set(selected_variables))
    pval = pd.Series(index=remainder, dtype='float64')  ## 변수의 p-value
    ## 기존에 포함된 변수와 새로운 변수 하나씩 돌아가면서
    ## 선형 모형을 적합한다.
    for col in remainder:
        y = df['RESULT']  ## 반응 변수
        X = df[selected_variables + [col]]
        X = sm.add_constant(X)
        model = sm.OLS(y, X).fit()
        pval[col] = model.pvalues[col]

    min_pval = pval.min()
    if min_pval < sl_enter:  ## 최소 p-value 값이 기준 값보다 작으면 포함
        selected_variables.append(pval.idxmin())
        ## 선택된 변수들에대해서
        ## 어떤 변수를 제거할지 고른다.
        while len(selected_variables) > 0:
            selected_X = df[selected_variables]
            selected_X = sm.add_constant(selected_X)
            selected_pval = sm.OLS(y, selected_X).fit().pvalues[1:]  ## 절편항의 p-value는 뺀다
            max_pval = selected_pval.max()
            if max_pval >= sl_remove:  ## 최대 p-value값이 기준값보다 크거나 같으면 제외
                remove_variable = selected_pval.idxmax()
                selected_variables.remove(remove_variable)
            else:
                break

        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y, sm.add_constant(df[selected_variables])).fit().rsquared_adj
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

sv_df = df[selected_variables]
sv_df = pd.concat([sv_df, df['RESULT']], axis=1)

<h4>2. 정규화 검정 및 독립 t 검정</h4>

- 정규화 검정: 자료의 값들과 표준정규점수와의 선형상관관계를 측정하여 표본이 정규 분포의 가정을 만족하는지 검정하는 방법
- 독립 t 검정: 두 집단의 평균 차이를 검증하기 위한 방법

In [9]:
result_list = []
Gdf = sv_df[sv_df['RESULT'] == 0]
NGdf = sv_df[sv_df['RESULT'] == 1]
min_value = min(len(Gdf), len(NGdf))

epochs = 500

for epoch in range(epochs):
    Sample_g = []
    Sample_ng = []
    
    g = Gdf.sample(n=min_value, replace=False, axis=0)
    for i, col in enumerate(g.columns):
        if col != 'RESULT':
            cnt = 0
            sample_g = g[col]

            if min_value <= 100:
                while (stats.ttest_1samp(sample_g, Gdf[col].mean()).pvalue > 0.05):
                    sample_g = Gdf[col].sample(n=min_value, replace=False)
            elif min_value > 100:
                while (stats.ttest_1samp(sample_g, Gdf[col].mean()).pvalue > 0.05):
                    sample_g = Gdf[col].sample(n=min_value, replace=False)

            Sample_g.append(sample_g)
            Sample_ng.append(NGdf.sample(n=min_value, replace=False, axis=0)[col])

    for i, col in enumerate(g.columns):
        if col != 'RESULT':
            # 정규화 검정 & 독립 t 검정
            if float(shapiro(Sample_ng[i]).pvalue) < 0.05 and float(shapiro(Sample_g[i]).pvalue) < 0.05:
                if wilcoxon(Sample_g[i], Sample_ng[i], zero_method='pratt').pvalue < 0.05:
                    result_list.append(Sample_g[i])
                    
res = pd.DataFrame(result_list).T.columns
unique, counts = np.unique(res, return_counts=True)
uniq_cnt_dict = dict(zip(unique, counts))

result_list = []
for col in uniq_cnt_dict:
    if uniq_cnt_dict[col] >= epochs / 2:
        result_list.append(col)
        
# 유효한 속성 추출 결과
print("유효한 속성 추출 결과")
print(result_list)

유효한 속성 추출 결과
['ApplyS', 'Cushion', 'DP001', 'DP003', 'HP2ENDTEMP', 'HPTransPos', 'HPTransPrs', 'INPUT', 'InjPress', 'InjSpeed', 'Moisture', 'OUTTEMP', 'RecoveryT', 'SPTEMP', 'Temp9', 'UvTemp1', 'UvTemp2', 'WH2INFLUX', 'WH3INTEMP', 'WH4INFLUX', 'WH4INTEMP', 'uvlamp_cycletime']


### 7. 유효한 속성을 기반한 feature(x), label(y) 데이터 분리

In [10]:
cols = result_list

x = df.loc[:, cols]
y = df.iloc[:, -1]

### 8. 정규화 수행
- StandardScaler를 사용한 정규화 수행

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x)

s_df = scaler.transform(x)
x = pd.DataFrame(s_df, columns=x.columns)
x

,Cushion,DP001,DP003,HP2ENDTEMP,HPTransPos,HPTransPrs,InjPress,InjSpeed,Moisture,OUTTEMP,RecoveryT,SPTEMP,Temp9,UvTemp1,UvTemp2,WH2INFLUX,WH3INTEMP,WH4INFLUX,WH4INTEMP
0,0.571450,-2.339533,0.558960,0.508697,0.643623,-0.546657,-0.752722,0.504621,-0.205244,-1.519669,0.364686,-3.718187,-2.107437,-2.076621,-1.967698,0.783651,0.460334,-0.031905,0.181869
1,0.552079,-1.988689,0.053671,0.508697,0.662337,-0.513607,-0.729409,0.504621,-0.205244,-1.499231,0.389593,-3.718187,-2.107437,-2.325270,-1.967698,0.804328,0.465436,-0.031905,0.177418
2,0.571450,-1.988689,-0.451619,0.508697,0.662337,-0.518329,-0.724747,0.504621,-0.205244,-1.478794,0.339779,-3.718187,-2.107437,-2.325270,-1.529787,0.793989,0.465436,-0.038306,0.177418
3,0.571450,-1.725555,0.053671,0.508697,0.662337,-0.537214,-0.748059,0.504621,-0.205244,-1.417483,0.377139,-3.718187,-2.107437,-2.822569,-1.967698,0.799159,0.460334,-0.031905,0.181869
4,0.571450,-1.813267,-0.451619,0.508697,0.662337,-0.560821,-0.762046,0.504621,-0.205244,-1.397046,0.379630,-3.660421,-2.107437,-2.822569,-1.748742,0.788820,0.460334,-0.035105,0.177418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9645,-2.101747,0.116379,-1.462197,-1.350393,-1.770531,-0.645805,0.510804,2.246012,-0.205244,1.402844,-1.941728,2.404933,1.413706,0.907171,0.878728,-0.813601,1.001104,2.343164,0.671445
9646,-2.140489,-0.234466,-1.462197,-1.350393,-1.770531,-0.636362,0.478166,2.187966,-0.205244,1.382407,-1.964144,2.462699,1.413706,0.658521,0.221860,-0.813601,1.001104,2.349566,0.631389
9647,-2.101747,0.116379,-0.956908,-1.350393,-1.789245,-0.683576,0.524791,2.246012,-0.205244,1.361970,-1.954181,2.462699,1.413706,0.658521,0.221860,-0.813601,0.996002,2.349566,0.626938
9648,-2.101747,0.116379,-0.956908,-1.350393,-1.789245,-0.674133,0.515466,2.246012,-0.205244,1.361970,-1.964144,2.462699,1.413706,0.907171,0.221860,-0.813601,0.996002,2.359169,0.622488


### 9. train-test split 수행
- train set:test set = 8:2 비율 분할
- 시계열 데이터이므로 'shuffle=False'를 통해 데이터가 섞이는 것을 방지

In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)

### 10. 모델 학습 및 결과 예측
- MO-GAAL : Generative Adversarial Active Learning for Unsupervised Outlier Detection 모델 사용
- 참고 자료: https://github.com/leibinghe/GAAL-based-outlier-detection

In [13]:
# MO_GAAL Model
from pyod.models.mo_gaal import MO_GAAL

mogaal = MO_GAAL(
    k=10,
    stop_epochs=15,
    lr_d=0.01,
    lr_g=0.0001,
    decay=1e-06,
    momentum=0.9,
    contamination=0.1,
)
mogaal.fit(x_train)

pred_mogaal = mogaal.predict(x_test)

print(classification_report(y_test, pred_mogaal))

Epoch 1 of 45

Testing for epoch 1 index 1:



Testing for epoch 1 index 2:

Testing for epoch 1 index 3:

Testing for epoch 1 index 4:

Testing for epoch 1 index 5:

Testing for epoch 1 index 6:

Testing for epoch 1 index 7:

Testing for epoch 1 index 8:

Testing for epoch 1 index 9:

Testing for epoch 1 index 10:

Testing for epoch 1 index 11:

Testing for epoch 1 index 12:

Testing for epoch 1 index 13:

Testing for epoch 1 index 14:

Testing for epoch 1 index 15:
Epoch 2 of 45

Testing for epoch 2 index 1:

Testing for epoch 2 index 2:

Testing for epoch 2 index 3:

Testing for epoch 2 index 4:

Testing for epoch 2 index 5:

Testing for epoch 2 index 6:

Testing for epoch 2 index 7:

Testing for epoch 2 index 8:

Testing for epoch 2 index 9:

Testing for epoch 2 index 10:

Testing for epoch 2 index 11:

Testing for epoch 2 index 12:

Testing for epoch 2 index 13:

Testing for epoch 2 index 14:

Testing for epoch 2 index 15:
Epoch 3 of 45

Testing for epoch 3 index 1:

Testing for epoch 3 index 2:

Testing for epoch 3 index 3:



Testing for epoch 13 index 10:

Testing for epoch 13 index 11:

Testing for epoch 13 index 12:

Testing for epoch 13 index 13:

Testing for epoch 13 index 14:

Testing for epoch 13 index 15:
Epoch 14 of 45

Testing for epoch 14 index 1:

Testing for epoch 14 index 2:

Testing for epoch 14 index 3:

Testing for epoch 14 index 4:

Testing for epoch 14 index 5:

Testing for epoch 14 index 6:

Testing for epoch 14 index 7:

Testing for epoch 14 index 8:

Testing for epoch 14 index 9:

Testing for epoch 14 index 10:

Testing for epoch 14 index 11:

Testing for epoch 14 index 12:

Testing for epoch 14 index 13:

Testing for epoch 14 index 14:

Testing for epoch 14 index 15:
Epoch 15 of 45

Testing for epoch 15 index 1:

Testing for epoch 15 index 2:

Testing for epoch 15 index 3:

Testing for epoch 15 index 4:

Testing for epoch 15 index 5:

Testing for epoch 15 index 6:

Testing for epoch 15 index 7:

Testing for epoch 15 index 8:

Testing for epoch 15 index 9:

Testing for epoch 15 index 

16/16 [==============================] - 0s 1ms/step - loss: 2.8522

Testing for epoch 17 index 8:
16/16 [==============================] - 0s 1ms/step - loss: 2.8873

Testing for epoch 17 index 9:
16/16 [==============================] - 0s 1ms/step - loss: 2.8962

Testing for epoch 17 index 10:
16/16 [==============================] - 0s 2ms/step - loss: 2.9057

Testing for epoch 17 index 11:
16/16 [==============================] - 0s 2ms/step - loss: 2.9184

Testing for epoch 17 index 12:
16/16 [==============================] - 0s 2ms/step - loss: 2.9309

Testing for epoch 17 index 13:
16/16 [==============================] - 0s 1ms/step - loss: 2.9378

Testing for epoch 17 index 14:
16/16 [==============================] - 0s 2ms/step - loss: 2.9377

Testing for epoch 17 index 15:
16/16 [==============================] - 0s 2ms/step - loss: 2.9620
Epoch 18 of 45

Testing for epoch 18 index 1:
16/16 [==============================] - 0s 2ms/step - loss: 2.9395

Testing for epoch 1

16/16 [==============================] - 0s 1ms/step - loss: 3.0586

Testing for epoch 18 index 15:
16/16 [==============================] - 0s 2ms/step - loss: 3.0903
Epoch 19 of 45

Testing for epoch 19 index 1:
16/16 [==============================] - 0s 1ms/step - loss: 3.0440

Testing for epoch 19 index 2:
16/16 [==============================] - 0s 2ms/step - loss: 3.1018

Testing for epoch 19 index 3:
16/16 [==============================] - 0s 2ms/step - loss: 3.0989

Testing for epoch 19 index 4:
16/16 [==============================] - 0s 1ms/step - loss: 3.1382

Testing for epoch 19 index 5:
16/16 [==============================] - 0s 2ms/step - loss: 3.1450

Testing for epoch 19 index 6:
16/16 [==============================] - 0s 2ms/step - loss: 3.1111

Testing for epoch 19 index 7:
16/16 [==============================] - 0s 2ms/step - loss: 3.1262

Testing for epoch 19 index 8:
16/16 [==============================] - 0s 2ms/step - loss: 3.1544

Testing for epoch 19 ind

...


Testing for epoch 39 index 13:
16/16 [==============================] - 0s 1ms/step - loss: 4.6465

Testing for epoch 39 index 14:
16/16 [==============================] - 0s 1ms/step - loss: 4.6524

Testing for epoch 39 index 15:
16/16 [==============================] - 0s 1ms/step - loss: 4.6797
Epoch 40 of 45

Testing for epoch 40 index 1:
16/16 [==============================] - 0s 1ms/step - loss: 4.6604

Testing for epoch 40 index 2:
16/16 [==============================] - 0s 1ms/step - loss: 4.6288

Testing for epoch 40 index 3:
16/16 [==============================] - 0s 1ms/step - loss: 4.6348

Testing for epoch 40 index 4:
16/16 [==============================] - 0s 1ms/step - loss: 4.6860

Testing for epoch 40 index 5:
16/16 [==============================] - 0s 1ms/step - loss: 4.6468

Testing for epoch 40 index 6:
16/16 [==============================] - 0s 1ms/step - loss: 4.6455

Testing for epoch 40 index 7:
16/16 [==============================] - 0s 1ms/step - loss:

16/16 [==============================] - 0s 1ms/step - loss: 4.6812

Testing for epoch 41 index 6:
16/16 [==============================] - 0s 1ms/step - loss: 4.7099

Testing for epoch 41 index 7:
16/16 [==============================] - 0s 1ms/step - loss: 4.7073

Testing for epoch 41 index 8:
16/16 [==============================] - 0s 1ms/step - loss: 4.7525

Testing for epoch 41 index 9:
16/16 [==============================] - 0s 1ms/step - loss: 4.7151

Testing for epoch 41 index 10:
16/16 [==============================] - 0s 1ms/step - loss: 4.6972

Testing for epoch 41 index 11:
16/16 [==============================] - 0s 1ms/step - loss: 4.7151

Testing for epoch 41 index 12:
16/16 [==============================] - 0s 1ms/step - loss: 4.6898

Testing for epoch 41 index 13:
16/16 [==============================] - 0s 1ms/step - loss: 4.7224

Testing for epoch 41 index 14:
16/16 [==============================] - 0s 2ms/step - loss: 4.7285

Testing for epoch 41 index 15:
16/1

16/16 [==============================] - 0s 1ms/step - loss: 4.6968
Epoch 42 of 45

Testing for epoch 42 index 1:
16/16 [==============================] - 0s 1ms/step - loss: 4.7363

Testing for epoch 42 index 2:
16/16 [==============================] - 0s 2ms/step - loss: 4.7196

Testing for epoch 42 index 3:
16/16 [==============================] - 0s 1ms/step - loss: 4.8096

Testing for epoch 42 index 4:
16/16 [==============================] - 0s 1ms/step - loss: 4.7474

Testing for epoch 42 index 5:
16/16 [==============================] - 0s 1ms/step - loss: 4.7406

Testing for epoch 42 index 6:
16/16 [==============================] - 0s 1ms/step - loss: 4.7122

Testing for epoch 42 index 7:
16/16 [==============================] - 0s 2ms/step - loss: 4.7218

Testing for epoch 42 index 8:
16/16 [==============================] - 0s 1ms/step - loss: 4.7567

Testing for epoch 42 index 9:
16/16 [==============================] - 0s 2ms/step - loss: 4.7695

Testing for epoch 42 inde

16/16 [==============================] - 0s 1ms/step - loss: 4.7721

Testing for epoch 43 index 8:
16/16 [==============================] - 0s 1ms/step - loss: 4.7969

Testing for epoch 43 index 9:
16/16 [==============================] - 0s 1ms/step - loss: 4.7838

Testing for epoch 43 index 10:
16/16 [==============================] - 0s 1ms/step - loss: 4.7702

Testing for epoch 43 index 11:
16/16 [==============================] - 0s 1ms/step - loss: 4.8324

Testing for epoch 43 index 12:
16/16 [==============================] - 0s 1ms/step - loss: 4.8162

Testing for epoch 43 index 13:
16/16 [==============================] - 0s 1ms/step - loss: 4.7431

Testing for epoch 43 index 14:
16/16 [==============================] - 0s 1ms/step - loss: 4.7612

Testing for epoch 43 index 15:
16/16 [==============================] - 0s 2ms/step - loss: 4.7962
Epoch 44 of 45

Testing for epoch 44 index 1:
16/16 [==============================] - 0s 1ms/step - loss: 4.8198

Testing for epoch 4

16/16 [==============================] - 0s 1ms/step - loss: 4.7796

Testing for epoch 44 index 15:
16/16 [==============================] - 0s 1ms/step - loss: 4.8279
Epoch 45 of 45

Testing for epoch 45 index 1:
16/16 [==============================] - 0s 1ms/step - loss: 4.8087

Testing for epoch 45 index 2:
16/16 [==============================] - 0s 1ms/step - loss: 4.8739

Testing for epoch 45 index 3:
16/16 [==============================] - 0s 1ms/step - loss: 4.8748

Testing for epoch 45 index 4:
16/16 [==============================] - 0s 1ms/step - loss: 4.8475

Testing for epoch 45 index 5:
16/16 [==============================] - 0s 1ms/step - loss: 4.8933

Testing for epoch 45 index 6:
16/16 [==============================] - 0s 1ms/step - loss: 4.8152

Testing for epoch 45 index 7:
16/16 [==============================] - 0s 1ms/step - loss: 4.8940

Testing for epoch 45 index 8:
16/16 [==============================] - 0s 1ms/step - loss: 4.8692

Testing for epoch 45 ind

### 예측 결과(Macro avg: f1-score) : 약 82%